In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore')

/Users/kunthshah/Desktop/Crypto-Price-Moevement-Prediction/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
df = pd.read_csv('Dataset/crypto_price_movement_dataset_with_lags.csv')
df.head()

,Date,Open_Price,Close_Price,High_Price,Low_Price,Price_Change,Volume,MA_5,MA_10,RSI,...,MA_10_lag_1,MA_10_lag_2,MA_10_lag_3,MA_10_lag_4,MA_10_lag_5,MA_10_lag_6,MA_10_lag_7,MA_10_lag_8,Momentum_ratios,Sentiment_deltas
0,2023-01-12,59097.295565,59029.726208,59900.004347,58784.035582,-67.569357,4462601,49045.482581,44580.256718,33.144940,...,44528.878721,45577.477694,0.000000,0.000000,0.000000,0.000000,0.0,0.0,1.898218,0.294423
1,2023-01-13,54973.279224,55151.317602,55396.713428,54378.735255,178.038378,9775141,48852.372428,44929.860092,43.650581,...,44580.256718,44528.878721,45577.477694,0.000000,0.000000,0.000000,0.0,0.0,0.934297,-1.048490
2,2023-01-14,36370.173320,36497.900856,36938.992350,36371.219308,127.727536,5726515,46570.552431,43760.010546,44.739048,...,44929.860092,44580.256718,44528.878721,45577.477694,0.000000,0.000000,0.0,0.0,0.661777,-1.083444
3,2023-01-15,35454.749016,35059.052567,35211.603012,34107.853810,-395.696449,4617507,43367.089528,43805.992068,37.650721,...,43760.010546,44929.860092,44580.256718,44528.878721,45577.477694,0.000000,0.0,0.0,0.960577,-1.199126
4,2023-01-16,35502.135296,35750.062833,36428.171405,35625.323065,247.927537,4181273,44297.612013,43903.552042,39.993981,...,43805.992068,43760.010546,44929.860092,44580.256718,44528.878721,45577.477694,0.0,0.0,1.019710,0.543907


In [8]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# 1. Split raw DataFrame by time (70/15/15)
n = len(df)
train_end = int(n * 0.70)
val_end   = train_end + int(n * 0.15)

train_df = df.iloc[:train_end].copy()
val_df   = df.iloc[train_end:val_end].copy()
test_df  = df.iloc[val_end:].copy()

# 2. Identify feature columns (exclude Date, target, and any leakage features)
feature_cols = [
    'Close_Price','Global_Economy','High_Price','Low_Price',
    'MA_10','MA_10_lag_2','Open_Price','RSI','Volatility'
    # add any engineered features here
]

# 3. Fit scaler on TRAIN only, then transform all splits
scaler = StandardScaler()
train_df[feature_cols] = scaler.fit_transform(train_df[feature_cols])
val_df[feature_cols]   = scaler.transform(val_df[feature_cols])
test_df[feature_cols]  = scaler.transform(test_df[feature_cols])

# 4. Function to build time‑windows
def create_time_windows(df, feature_cols, window_size=10):
    X, y = [], []
    for i in range(window_size, len(df)):
        X.append(df[feature_cols].iloc[i-window_size:i].values)
        y.append(df['Price_Movement'].iloc[i])
    return np.array(X), np.array(y)

# 5. Create windowed datasets
window_size = 10
X_train, y_train = create_time_windows(train_df, feature_cols, window_size)
X_val,   y_val   = create_time_windows(val_df,   feature_cols, window_size)
X_test,  y_test  = create_time_windows(test_df,  feature_cols, window_size)

print("Shapes:",
      X_train.shape, y_train.shape,
      X_val.shape,   y_val.shape,
      X_test.shape,  y_test.shape)


Shapes: (34982, 10, 9) (34982,) (7488, 10, 9) (7488,) (7489, 10, 9) (7489,)


# Attention on top of LSTM

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, LSTM, Dense, Dropout, BatchNormalization, 
    Permute, Multiply, Lambda, Softmax, Concatenate
)

def attention_block(inputs):
    # inputs.shape = (batch, timesteps, features)
    # 1) Learn a score for each timestep
    a = Permute((2,1))(inputs)                       # (batch, features, timesteps)
    a = Dense(inputs.shape[1], activation='tanh')(a) # (batch, features, timesteps)
    a = Softmax(axis=-1)(a)                          # normalize over timesteps
    a_probs = Permute((2,1))(a)                      # (batch, timesteps, features)
    # 2) Weight the inputs
    output = Multiply()([inputs, a_probs])           # (batch, timesteps, features)
    # 3) Sum across time
    return Lambda(lambda x: tf.reduce_sum(x, axis=1))(output)

def create_lstm_with_attention(timesteps, n_features):
    inp = Input(shape=(timesteps, n_features))
    
    x = LSTM(64, return_sequences=True)(inp)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    x = LSTM(32, return_sequences=True)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    # Attention pooling
    x = attention_block(x)   # (batch, features)
    
    x = Dense(16, activation='relu')(x)
    x = Dropout(0.2)(x)
    out = Dense(1, activation='sigmoid')(x)
    
    model = Model(inp, out)
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['AUC','accuracy']
    )
    return model

model_attn = create_lstm_with_attention(X_train.shape[1], X_train.shape[2])
model_attn.summary()


Model: "functional_27"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 10, 9)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_6 (LSTM)       │ (None, 10, 64)    │     18,944 │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 10, 64)    │        256 │ lstm_6[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_9 (Dropout) │ (None, 10, 64)    │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_7 (LSTM)       │ (None, 10, 32)    │     12,416 │ dropout_9[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 10, 32)    │        128 │ lstm_7[0][0]      │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_10          │ (None, 10, 32)    │          0 │ batch_normalizat… │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ permute (Permute)   │ (None, 32, 10)    │          0 │ dropout_10[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 32, 10)    │        110 │ permute[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ softmax (Softmax)   │ (None, 32, 10)    │          0 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ permute_1 (Permute) │ (None, 10, 32)    │          0 │ softmax[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 10, 32)    │          0 │ dropout_10[0][0], │
│                     │                   │            │ permute_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 32)        │          0 │ multiply[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 16)        │        528 │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_11          │ (None, 16)        │          0 │ dense_7[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 1)         │         17 │ dropout_11[0][0]  │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 32,399 (126.56 KB)

 Trainable params: 32,207 (125.81 KB)

 Non-trainable params: 192 (768.00 B)

In [10]:
# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6),
    ModelCheckpoint('best_lstm_model.h5', monitor='val_loss', save_best_only=True)
]

# Train
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=64,
    callbacks=callbacks,
    verbose=2
)

Epoch 1/100


547/547 - 5s - 8ms/step - AUC: 0.5009 - accuracy: 0.5023 - loss: 0.6932 - val_AUC: 0.4917 - val_accuracy: 0.4959 - val_loss: 0.6934 - learning_rate: 1.0000e-03
Epoch 2/100


547/547 - 4s - 7ms/step - AUC: 0.5046 - accuracy: 0.5046 - loss: 0.6931 - val_AUC: 0.5033 - val_accuracy: 0.4977 - val_loss: 0.6932 - learning_rate: 1.0000e-03
Epoch 3/100
547/547 - 3s - 6ms/step - AUC: 0.5002 - accuracy: 0.5015 - loss: 0.6932 - val_AUC: 0.5069 - val_accuracy: 0.4981 - val_loss: 0.6932 - learning_rate: 1.0000e-03
Epoch 4/100
547/547 - 4s - 7ms/step - AUC: 0.4945 - accuracy: 0.5028 - loss: 0.6932 - val_AUC: 0.5003 - val_accuracy: 0.4988 - val_loss: 0.6932 - learning_rate: 1.0000e-03
Epoch 5/100


547/547 - 4s - 7ms/step - AUC: 0.5003 - accuracy: 0.5034 - loss: 0.6931 - val_AUC: 0.5015 - val_accuracy: 0.4979 - val_loss: 0.6932 - learning_rate: 1.0000e-03
Epoch 6/100


547/547 - 3s - 6ms/step - AUC: 0.5026 - accuracy: 0.5030 - loss: 0.6931 - val_AUC: 0.5020 - val_accuracy: 0.4983 - val_loss: 0.6932 - learning_rate: 1.0000e-03
Epoch 7/100


...

547/547 - 3s - 6ms/step - AUC: 0.5055 - accuracy: 0.5038 - loss: 0.6930 - val_AUC: 0.5061 - val_accuracy: 0.4981 - val_loss: 0.6931 - learning_rate: 2.5000e-04
Epoch 15/100
547/547 - 3s - 6ms/step - AUC: 0.5052 - accuracy: 0.5037 - loss: 0.6929 - val_AUC: 0.5006 - val_accuracy: 0.4983 - val_loss: 0.6934 - learning_rate: 2.5000e-04
Epoch 16/100
547/547 - 3s - 6ms/step - AUC: 0.5019 - accuracy: 0.5045 - loss: 0.6930 - val_AUC: 0.5003 - val_accuracy: 0.4983 - val_loss: 0.6933 - learning_rate: 2.5000e-04
Epoch 17/100


547/547 - 3s - 6ms/step - AUC: 0.5037 - accuracy: 0.5031 - loss: 0.6930 - val_AUC: 0.5050 - val_accuracy: 0.4983 - val_loss: 0.6931 - learning_rate: 2.5000e-04
Epoch 18/100


547/547 - 3s - 6ms/step - AUC: 0.5066 - accuracy: 0.5044 - loss: 0.6929 - val_AUC: 0.5058 - val_accuracy: 0.4985 - val_loss: 0.6929 - learning_rate: 2.5000e-04
Epoch 19/100
547/547 - 3s - 6ms/step - AUC: 0.5077 - accuracy: 0.5045 - loss: 0.6927 - val_AUC: 0.5036 - val_accuracy: 0.5007 - val_loss: 0.6929 - learning_rate: 2.5000e-04
Epoch 20/100


547/547 - 3s - 6ms/step - AUC: 0.5083 - accuracy: 0.5065 - loss: 0.6929 - val_AUC: 0.5068 - val_accuracy: 0.5019 - val_loss: 0.6929 - learning_rate: 2.5000e-04
Epoch 21/100
547/547 - 3s - 6ms/step - AUC: 0.5079 - accuracy: 0.5047 - loss: 0.6928 - val_AUC: 0.4990 - val_accuracy: 0.4993 - val_loss: 0.6932 - learning_rate: 2.5000e-04
Epoch 22/100
547/547 - 3s - 6ms/step - AUC: 0.5089 - accuracy: 0.5084 - loss: 0.6929 - val_AUC: 0.5026 - val_accuracy: 0.5033 - val_loss: 0.6931 - learning_rate: 2.5000e-04
Epoch 23/100
547/547 - 3s - 6ms/step - AUC: 0.5111 - accuracy: 0.5066 - loss: 0.6928 - val_AUC: 0.5098 - val_accuracy: 0.5000 - val_loss: 0.6929 - learning_rate: 2.5000e-04
Epoch 24/100
547/547 - 3s - 6ms/step - AUC: 0.5128 - accuracy: 0.5065 - loss: 0.6926 - val_AUC: 0.5063 - val_accuracy: 0.4984 - val_loss: 0.6929 - learning_rate: 1.2500e-04
Epoch 25/100
547/547 - 3s - 6ms/step - AUC: 0.5080 - accuracy: 0.5045 - loss: 0.6927 - val_AUC: 0.5050 - val_accuracy: 0.4979 - val_loss: 0.6929 - l

547/547 - 3s - 6ms/step - AUC: 0.5122 - accuracy: 0.5071 - loss: 0.6926 - val_AUC: 0.5088 - val_accuracy: 0.5012 - val_loss: 0.6929 - learning_rate: 1.2500e-04
Epoch 27/100
547/547 - 3s - 6ms/step - AUC: 0.5107 - accuracy: 0.5077 - loss: 0.6926 - val_AUC: 0.5074 - val_accuracy: 0.4964 - val_loss: 0.6930 - learning_rate: 1.2500e-04
Epoch 28/100
547/547 - 3s - 6ms/step - AUC: 0.5131 - accuracy: 0.5067 - loss: 0.6923 - val_AUC: 0.5098 - val_accuracy: 0.5028 - val_loss: 0.6929 - learning_rate: 1.2500e-04
Epoch 29/100
547/547 - 3s - 6ms/step - AUC: 0.5155 - accuracy: 0.5089 - loss: 0.6925 - val_AUC: 0.5065 - val_accuracy: 0.5000 - val_loss: 0.6929 - learning_rate: 6.2500e-05
Epoch 30/100
547/547 - 3s - 6ms/step - AUC: 0.5107 - accuracy: 0.5097 - loss: 0.6925 - val_AUC: 0.5072 - val_accuracy: 0.4971 - val_loss: 0.6931 - learning_rate: 6.2500e-05
Epoch 31/100
547/547 - 3s - 6ms/step - AUC: 0.5138 - accuracy: 0.5076 - loss: 0.6924 - val_AUC: 0.5059 - val_accuracy: 0.4977 - val_loss: 0.6929 - l

# CNN with LSTM

In [12]:
from tensorflow.keras.layers import Conv1D, MaxPooling1D

def create_cnn_lstm(timesteps, n_features):
    inp = Input(shape=(timesteps, n_features))
    
    # Convolutional front‑end
    x = Conv1D(64, kernel_size=3, activation='relu', padding='same')(inp)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.2)(x)
    
    x = Conv1D(32, kernel_size=3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.2)(x)
    
    # LSTM back‑end
    x = LSTM(32)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    x = Dense(16, activation='relu')(x)
    x = Dropout(0.2)(x)
    out = Dense(1, activation='sigmoid')(x)
    
    model = Model(inp, out)
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['AUC','accuracy']
    )
    return model

model_cnn_lstm = create_cnn_lstm(X_train.shape[1], X_train.shape[2])
model_cnn_lstm.summary()

Model: "functional_29"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 10, 9)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 10, 64)         │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 10, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 5, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 5, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_3 (Conv1D)               │ (None, 5, 32)          │         6,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 5, 32)          │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 2, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 2, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ (None, 32)             │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 17,345 (67.75 KB)

 Trainable params: 17,089 (66.75 KB)

 Non-trainable params: 256 (1.00 KB)

In [13]:
# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6),
    ModelCheckpoint('best_lstm_model.h5', monitor='val_loss', save_best_only=True)
]

# Train
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=64,
    callbacks=callbacks,
    verbose=2
)

Epoch 1/100


547/547 - 4s - 6ms/step - AUC: 0.5104 - accuracy: 0.5056 - loss: 0.6925 - val_AUC: 0.5072 - val_accuracy: 0.5011 - val_loss: 0.6929 - learning_rate: 3.1250e-05
Epoch 2/100
547/547 - 3s - 6ms/step - AUC: 0.5106 - accuracy: 0.5051 - loss: 0.6926 - val_AUC: 0.5096 - val_accuracy: 0.5001 - val_loss: 0.6930 - learning_rate: 3.1250e-05
Epoch 3/100


547/547 - 4s - 7ms/step - AUC: 0.5139 - accuracy: 0.5102 - loss: 0.6926 - val_AUC: 0.5098 - val_accuracy: 0.4975 - val_loss: 0.6929 - learning_rate: 3.1250e-05
Epoch 4/100


547/547 - 3s - 6ms/step - AUC: 0.5148 - accuracy: 0.5083 - loss: 0.6924 - val_AUC: 0.5107 - val_accuracy: 0.4977 - val_loss: 0.6929 - learning_rate: 3.1250e-05
Epoch 5/100
547/547 - 4s - 7ms/step - AUC: 0.5104 - accuracy: 0.5067 - loss: 0.6927 - val_AUC: 0.5123 - val_accuracy: 0.4980 - val_loss: 0.6929 - learning_rate: 3.1250e-05
Epoch 6/100


547/547 - 3s - 6ms/step - AUC: 0.5114 - accuracy: 0.5060 - loss: 0.6925 - val_AUC: 0.5097 - val_accuracy: 0.5000 - val_loss: 0.6929 - learning_rate: 3.1250e-05
Epoch 7/100
547/547 - 3s - 6ms/step - AUC: 0.5115 - accuracy: 0.5043 - loss: 0.6924 - val_AUC: 0.5090 - val_accuracy: 0.4983 - val_loss: 0.6929 - learning_rate: 1.5625e-05
Epoch 8/100
547/547 - 3s - 6ms/step - AUC: 0.5135 - accuracy: 0.5069 - loss: 0.6925 - val_AUC: 0.5087 - val_accuracy: 0.4989 - val_loss: 0.6929 - learning_rate: 1.5625e-05
Epoch 9/100


547/547 - 3s - 6ms/step - AUC: 0.5149 - accuracy: 0.5087 - loss: 0.6924 - val_AUC: 0.5078 - val_accuracy: 0.4996 - val_loss: 0.6929 - learning_rate: 1.5625e-05
Epoch 10/100
547/547 - 3s - 6ms/step - AUC: 0.5112 - accuracy: 0.5075 - loss: 0.6925 - val_AUC: 0.5082 - val_accuracy: 0.4996 - val_loss: 0.6929 - learning_rate: 1.5625e-05
Epoch 11/100
547/547 - 3s - 6ms/step - AUC: 0.5135 - accuracy: 0.5088 - loss: 0.6924 - val_AUC: 0.5082 - val_accuracy: 0.4987 - val_loss: 0.6929 - learning_rate: 1.5625e-05
Epoch 12/100
547/547 - 3s - 6ms/step - AUC: 0.5136 - accuracy: 0.5087 - loss: 0.6925 - val_AUC: 0.5098 - val_accuracy: 0.4987 - val_loss: 0.6929 - learning_rate: 7.8125e-06
Epoch 13/100
547/547 - 3s - 6ms/step - AUC: 0.5151 - accuracy: 0.5098 - loss: 0.6924 - val_AUC: 0.5087 - val_accuracy: 0.4983 - val_loss: 0.6929 - learning_rate: 7.8125e-06
Epoch 14/100
547/547 - 3s - 6ms/step - AUC: 0.5130 - accuracy: 0.5083 - loss: 0.6923 - val_AUC: 0.5089 - val_accuracy: 0.4993 - val_loss: 0.6929 - l

In [17]:
# Install keras-tcn if needed: pip install keras-tcn
from tcn import TCN

def create_tcn(timesteps, n_features):
    inp = Input(shape=(timesteps, n_features))
    x = TCN(nb_filters=64, kernel_size=3, dilations=[1,2,4,8],
            return_sequences=False, activation='relu')(inp)
    x = Dropout(0.2)(x)
    
    x = Dense(16, activation='relu')(x)
    x = Dropout(0.2)(x)
    out = Dense(1, activation='sigmoid')(x)
    
    model = Model(inp, out)
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['AUC','accuracy']
    )
    return model

model_tcn = create_tcn(X_train.shape[1], X_train.shape[2])
model_tcn.summary()


Model: "functional_30"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 10, 9)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ tcn (TCN)                       │ (None, 64)             │        88,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 16)             │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 89,953 (351.38 KB)

 Trainable params: 89,953 (351.38 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6),
    ModelCheckpoint('best_lstm_model.h5', monitor='val_loss', save_best_only=True)
]

# Train
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=64,
    callbacks=callbacks,
    verbose=2
)

Epoch 1/100


547/547 - 4s - 6ms/step - AUC: 0.5129 - accuracy: 0.5103 - loss: 0.6922 - val_AUC: 0.5074 - val_accuracy: 0.4996 - val_loss: 0.6929 - learning_rate: 3.9063e-06
Epoch 2/100


547/547 - 3s - 6ms/step - AUC: 0.5166 - accuracy: 0.5111 - loss: 0.6923 - val_AUC: 0.5082 - val_accuracy: 0.4980 - val_loss: 0.6929 - learning_rate: 3.9063e-06
Epoch 3/100
547/547 - 3s - 6ms/step - AUC: 0.5144 - accuracy: 0.5081 - loss: 0.6924 - val_AUC: 0.5087 - val_accuracy: 0.4985 - val_loss: 0.6929 - learning_rate: 3.9063e-06
Epoch 4/100


547/547 - 3s - 6ms/step - AUC: 0.5107 - accuracy: 0.5096 - loss: 0.6923 - val_AUC: 0.5082 - val_accuracy: 0.4983 - val_loss: 0.6929 - learning_rate: 3.9063e-06
Epoch 5/100
547/547 - 3s - 6ms/step - AUC: 0.5147 - accuracy: 0.5085 - loss: 0.6925 - val_AUC: 0.5075 - val_accuracy: 0.4989 - val_loss: 0.6929 - learning_rate: 3.9063e-06
Epoch 6/100
547/547 - 3s - 6ms/step - AUC: 0.5161 - accuracy: 0.5080 - loss: 0.6923 - val_AUC: 0.5082 - val_accuracy: 0.4988 - val_loss: 0.6929 - learning_rate: 3.9063e-06
Epoch 7/100
547/547 - 3s - 6ms/step - AUC: 0.5171 - accuracy: 0.5131 - loss: 0.6923 - val_AUC: 0.5091 - val_accuracy: 0.4991 - val_loss: 0.6929 - learning_rate: 1.9531e-06
Epoch 8/100
547/547 - 3s - 6ms/step - AUC: 0.5145 - accuracy: 0.5069 - loss: 0.6924 - val_AUC: 0.5082 - val_accuracy: 0.4989 - val_loss: 0.6929 - learning_rate: 1.9531e-06
Epoch 9/100
547/547 - 3s - 6ms/step - AUC: 0.5153 - accuracy: 0.5111 - loss: 0.6923 - val_AUC: 0.5083 - val_accuracy: 0.4987 - val_loss: 0.6929 - learni